In [ ]:
# Load images, predictions, labels
"""
Here is an example use case of our system.
To use this example, ou need to get celebA dataset first. At http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

all_datas: a list of image address
labels: a list of labels
predictions: a list of predictions
split: a list of tags, tag can be {TRAIN, VALID, UNLABELED}
The above four list should have equal length
"""
import numpy as np
from utils.defines import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
f = open("exampleData/celeba/list_attr_celeba.txt")
lines = list(f.readlines())
lines = [line.strip('\n').split(' ')[0] for line in lines]
all_datas = ['../img_align_celeba/' + line for line in lines][1:]
train_idx = np.load("exampleData/celebA/train_idx.npy")[:80000]
valid_idx = np.arange(len(all_datas))[-100000:-80000]
unlabel_idx = np.arange(len(all_datas))[-80000:]
print(len(all_datas), len(train_idx), len(valid_idx), len(unlabel_idx))
idxs = np.concatenate([train_idx, valid_idx, unlabel_idx], axis=0)
labels = np.load("exampleData/celebA/labels.npy")[idxs]
predictions = np.load("exampleData/celebA/predictions.npy")[idxs]
all_datas = [all_datas[i] for i in idxs]
split = [TRAIN for i in  train_idx] + [VALID for i in  valid_idx] + [UNLABELED for i in  unlabel_idx]

In [ ]:
# Chat with our AI assistant, get visual attributes
# Alternatively, you can propose the visual attributes by yourself. Then skip this step.
from utils.gpt import ChatBot
api_key = "" # Your GPT api key
ChatBot = ChatBot(api_key)
ChatBot.chat() # chat with our AI assistent, you can stop when you satisfied with the attributes in the form

In [ ]:
"""
Create a corpus
We provide an example of corpus in exampleData/celebA/corpus.json, you can skip this step
"""
from utils.gpt import get_des_question, get_binary_question,get_des_prompt, get_type
from processData.Corpus import emptyAttribute
import json
attributes = ChatBot.attributes # a list of attributes: ["attribute1", "attribute2", "attribute3"]
attribute_dict = {}
main_object = "human"
for name in attributes:
    attribute_dict[name] = emptyAttribute()
    data_type = get_type(main_object,name)
    attribute_dict[name]['type'] = data_type
    if data_type == 'binary':
        attribute_dict[name]['question'] = get_binary_question(main_object,name)
    else:
        attribute_dict[name]['question'] = get_des_question(main_object,name)
        attribute_dict[name]['prompt'] = [get_des_prompt(main_object,name),]

# Although GPT is amazing, it is better to double-check the corpus base
for key, content in attribute_dict.items():
    print(key, content)

f = open("exampleData/celebA/corpus.json" ,'w')
f.write(json.dumps(attribute_dict,indent=4))

In [ ]:
"""
Now we can assign the value under each attribute for every data
We provide an example of attribute values in exampleData/celebA/attribute.json, you can skip this step
"""
from processData.AttributeSelection import AttributeSelection
AS = AttributeSelection(all_datas, labels, corpus_path="exampleData/celebA/corpus.json")
AS.match_description_to_data()
AS.save_attributes("exampleData/celeba/attribute.json")

In [ ]:
"""
Diagnose the model based on 1. attributes, 2. label, 3. predictions
"""
# # Diagnose
from processData.Diagnose import ModelDiagnose
MD = ModelDiagnose(labels, predictions, split, "exampleData/celeba/attribute.json")
MD.detect_failure_by_label()
MD.detect_prediction_correlation()
MD.detect_failure_prediction_correlation()

In [ ]:
"""
UI interface, explore the distribution of data, attributes
Download the dataset before use it
"""
embeddings = np.load("exampleData/celebA/2Dfeatures.npy")
embeddings = embeddings[idxs]
from interaction.interaction import Interaction
interaction = Interaction(
   datas=all_datas,
   labels=labels,
   predictions=predictions,
   embeddings=embeddings,
   tags=split,
   attributes_dict_path="exampleData/celebA/attribute.json"
)
interaction.interaction()